### *STEP1:* 将原来6G的xml文件切分成平均1M的小文档

In [1]:
RUN = 0
if(RUN):  
    fin = open('zhwiki-20171020-pages-articles-multistream.xml',encoding='utf8')
    for i in range(10000000):      
        readin = fin.readlines(1000*1000)
        if(not readin):
            break
        fout = open('parts/part_'+str(i),'w',encoding = 'utf8')
        fout.writelines(readin)
        # fout.writelines(re.sub("[\[\]\'\"\@#$%^&*()<>：{}|/_.+-—，,。!！ ？、~@#￥%……&*（）]+", "",s))
        fout.close()
    fin.close()
    partsNum = i
    print(partsNum)

**FUNCTION** 判断一个unicode是否为汉字

In [2]:
def is_ch(uchar):
        """判断一个unicode是否是汉字"""
        if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
                return True
        else:
                return False

### *STEP2:* 导入GBK编码表，构造一个dic，将汉字映射到整数

In [3]:
with open('GBK',encoding='utf8') as fin:
    chList = []
    for line in fin.readlines():
        for word in line:
            if(is_ch(word)):
                chList.append(word)

                
chNum = len(chList)
print('词典大小：',chNum)
ch2int = {}
idx = 0
with open('ch2int','w',encoding='utf8') as fout:  
    for w in chList:
        ch2int[w] = idx
        fout.write(w+'\t'+str(idx)+'\n')
        idx += 1


词典大小： 20902


**CLASS** 定义类BitSet 用于集合运算

In [4]:
class BitSet:
    def __init__(self, elementNum):
        self.bytesNum = round(elementNum/8)
        self.set = bytearray(self.bytesNum)
        
    def insert(self, index):
        if(index >= 0 and index>>3 < self.bytesNum):
            self.set[index >> 3] = self.set[index >> 3] | (1 << int(index & 7)) 
            return True;
        return False;
    
    def isElement(self, index):
        if(index >= 0 and index>>3 < self.bytesNum and (self.set[index >> 3] & (1<<(index & 7)))):
            return True
        return False
        
    def AND(self, bs):
        a = BitSet(self.bytesNum*8)
        for i in range(self.bytesNum):
            a.set[i] = self.set[i] & bs.set[i]
        return a

### *STEP3:* 给每一个汉字创建倒排索引，对应包含该汉字的文档

In [5]:
partsNum = 4919
bytesNum = round(partsNum/8)

invertedIndex = [BitSet(partsNum) for i in range(chNum)]
print('每个字的倒排索引表的字节数：',invertedIndex[0].bytesNum)

每个字的倒排索引表的字节数： 615


**FUNCTION** 显示进度条

In [6]:
import sys,time

def processBar(index, totalNum):
    bar = '#'*int((index/totalNum)*50)
    sys.stdout.write(str(int((index/totalNum)*100))+'%  ||'+bar+'->'+"\r")
    #sys.stdout.flush()

遍历所有文档的所有字符，建立倒排索引

In [7]:
RUN_iidx = 0
if(RUN_iidx):
    for partIdx in range(partsNum):

        processBar(partIdx, partsNum)

        with open('parts/part_'+str(partIdx),encoding='utf8') as f:
            for word in f.read():
                try:
                    if(is_ch(word)):
                        invertedIndex[ch2int[word]].insert(partIdx)
                except KeyError as err:
                    print('KEY ERROR'+str(err))

    for chIdx in range(chNum):    
        processBar(chIdx, chNum)
        with open('iidx/ch_'+str(chIdx)+'.iidx','w',encoding='utf8') as f:
                for j in invertedIndex[chIdx].set:
                    f.write(str(j)+' ')

测试：读取倒排索引

In [8]:
def readIidx(file):
    with open(file,encoding='utf8') as f:
        s = f.read()[:-1].split(' ')
    bSet = BitSet(len(s)*8)
    for i in range(bSet.bytesNum):
        bSet.set[i] = int(s[i])
    return bSet

"""
for chIdx in range(chNum):
    processBar(chIdx,chNum)
    f = 'iidx/ch_'+str(chIdx)+'.iidx'
    invertedIndex[chIdx] = readIidx(f)
    
print(invertedIndex[0].set[:20])
"""



"\nfor chIdx in range(chNum):\n    processBar(chIdx,chNum)\n    f = 'iidx/ch_'+str(chIdx)+'.iidx'\n    invertedIndex[chIdx] = readIidx(f)\n    \nprint(invertedIndex[0].set[:20])\n"

### *STEP4:* 建立每个文档的 tf 表

In [9]:
# tf 
RUN_tf = 1

if(RUN_tf):
    for partIdx in range(partsNum):

        tf = [0.0 for i in range(chNum)]
        total = 0

        with open('parts/part_'+str(partIdx),encoding='utf8') as f:
            for word in f.read():
                if(is_ch(word)):
                    tf[ch2int[word]] += 1
                    total += 1

        total = max(total, 1)
        for i in range(chNum):
            tf[i] /= total

        with open('parts/tf/part_'+str(partIdx)+'.tf','w',encoding='utf8') as fout:
            for i in tf:
                fout.write(str(i)+' ')
                
        processBar(partIdx, partsNum)


### *STEP5:* 计算每个单词的 idf 值

In [10]:
# idf(ch) = log(total DocumentNum / Num of documents containing ch)
import math

RUN_idf = 1
if(RUN_idf):
    
    idf = [0.0 for i in range(chNum)]

    for chIdx in range(chNum):
        processBar(chIdx, chNum)
        num = 0
        iidxFile = 'iidx/ch_'+str(chIdx)+'.iidx'
        bs = readIidx(iidxFile)
        for i in range(partsNum):
            if(bs.isElement(i)):
                num += 1
        idf[chIdx] = math.log(partsNum/(num+1)) # divide by zero

    with open('iidx/idf','w',encoding='utf8') as fout:
        for i in idf:
            fout.write(str(i)+' ')
    

读取每篇文档每个字的 tfidf 值

In [123]:
idfFile = 'iidx/idf'
with open(idfFile, encoding='utf8') as f:
    s = f.read().split()
IDF = [float(idf) for idf in s]

import numpy as np

ENOUGHMEMORY = True
if(ENOUGHMEMORY):
    TFIDF = np.zeros([partsNum,chNum])
    for partIdx in range(partsNum):
        processBar(partIdx, partsNum)
        file = 'parts/tf/part_'+str(partIdx)+'.tf'
        with open(file,encoding='utf8') as f:
            s = f.read().split()    
        for chIdx in range(chNum):
            TFIDF[partIdx][chIdx] = float(s[chIdx])*IDF[chIdx]
        


### *STEP6:* 搜索到包含关键词的文档集, 将搜索到的文档集根据 tf-idf 值排序

In [130]:
def mask(bytes):
    r = BitSet(bytes*8)
    for i in range(bytes):
        r.set[i] = 255
    return r

bsMask = mask(bytesNum) # 掩码

def tfidf(partIdx, keyList):
    
    if(ENOUGHMEMORY):
        return  sum([TFIDF[partIdx][chIdx] for chIdx in keyList])
    
    tfidf = 0
    file = 'parts/tf/part_'+str(partIdx)+'.tf'
    with open(file,encoding='utf8') as f:
        s = f.read().split()
        for i in keyList:
            tfidf += float(s[i])*IDF(i) 
    return tfidf

def search(query):
    keyList = []
    targets = bsMask
    for word in query:
        if(is_ch(word)):
            keyList.append(ch2int[word])
            targets = targets.AND(readIidx('iidx/ch_'+str(ch2int[word])+'.iidx'))
            
    result = []
    for i in range(partsNum):
        if(targets.isElement(i)):
            result.append(i)

    # rearrange
    tuples = []
    for doc in result:
        tuples.append((doc, tfidf(doc,keyList)))
    
    result = sorted(tuples, key = lambda x:x[1], reverse=1)
        
    return result


In [163]:
r = search('降龙十八掌')[:100]
print("（文档号，TFIDF值）")
for i in r:
    print(i)

（文档号，TFIDF值）
(2195, 0.0012538194937865626)
(2192, 0.00074516821934332783)
(3335, 0.00070944251423168069)
(3334, 0.00053639815331373464)
(2191, 0.00048238345187170538)
(2554, 0.00040487345298292432)
(3336, 0.00040323815270376802)
(4891, 0.00040130384277274974)
(1225, 0.00037096059308915096)
(4021, 0.0003702145265853077)
(1746, 0.00036683432065730661)
(2698, 0.00036536644465616285)
(2541, 0.00036071489111920478)
(2280, 0.00035553127974912075)
(2279, 0.00034865903782981938)
(2546, 0.00034503278985721354)
(3508, 0.00034479396238077525)
(2196, 0.00034082480195172452)
(3254, 0.00032731985297806482)
(2983, 0.00031677609220833982)
(336, 0.0003128326008106595)
(3258, 0.00031002191307890589)
(3327, 0.00030376847836756685)
(4906, 0.00029932409207689028)
(3903, 0.00029883267483686002)
(2199, 0.00028384795608407014)
(1945, 0.00028228236669312597)
(4904, 0.00028202085431423379)
(1608, 0.00027640804007445993)
(4900, 0.00027293930198780904)
(4869, 0.00027201629958735067)
(1394, 0.00026877560725819047)

假如每次只取出前十名，每篇文档大小1M，总共只占用10M内存，完全可以将所有文字体取出来分析，找到最佳匹配的句子

In [1]:
query = '降龙十八掌'
docList = [item[0] for item in search(query)[:100]]
s = []
for docIdx in docList:
    with open('parts/part_'+str(docIdx),encoding='utf8') as f:
        for line in f.readlines():
            check = True # 只有当所有关键词都在一句话中时为 True
            for word in query:
                if(word not in line):
                    check = False
                    break
            if(check):
                s.append('part_'+str(docIdx)+'   '+line)
        
print('前百篇文档的有效行数:',len(s))
print('包含所有关键词的句段如下：')
s[:10]

NameError: name 'search' is not defined

测试：繁简转换

In [162]:
from langconv import *

line = '為当今世界著名的[[美國]][[網球]]女子巨星，網球史上最伟大的女子选手之一，\
    美國體育史上最佳女子網球选手（與[[克里斯·埃弗特|Chris Evert]]並列），\
    全世界第一位網球大滿貫黑人女單冠軍，單打最高世界排名第一，\
    23座大滿貫女單冠軍得主，網球史上第二位女子單打生涯全滿貫得主。'
# 转换繁体到简体
line = Converter('zh-hans').convert(line)
print(line)
# 转换简体到繁体
line = Converter('zh-hant').convert(line)
print(line)

ModuleNotFoundError: No module named 'langconv'